In [1]:
import pandas as pd
import json 
from collections import defaultdict
import firebase_admin
from firebase_admin import credentials, auth 
from firebase_admin import db, firestore 
import datetime
import numpy as np

<h1>Firebase
 

In [2]:
cred = credentials.Certificate(r"D:\Workstuff\my-work-python-script\Database_in_pandas\Firebase_Shipment_Database\json_secret\logistec-firebase-adminsdk-mz58b-63b513fce5.json")
firebase_admin.initialize_app(cred, {})

firestore_db = firestore.client()

<h1>UTIL_49

In [63]:
#read clipboard
data = pd.read_clipboard(sep='\t')

#create new dataframe with selected columns
stockout_id = data["ID"].astype(str).str.replace(",","")
branchid = data["Branch (ID)"].astype(str).str.replace(",","")
branchname = data["Branch (Name)"]
phyid = pd.DataFrame(stockout_id + "-" + branchid).rename(columns={0: "PHYID"}, inplace=False)
docdate = pd.to_datetime(data["Doc Date"], dayfirst=True).dt.strftime("%d/%m/%y")
productcode = data["Product (Code)"].astype(str)
productname = data["Product (Name)"]
totalnumber = data["Number"]
cat_name = data["Category Name"]
serial = data["Serial"].fillna(data.pop('Product (Code)')).astype(str)

frames = [docdate,branchname, phyid, productcode, productname, serial, totalnumber, cat_name ]
new_frames = pd.concat(frames, axis=1)

#grouping every columns again to sum all duplicates + set new index.
grouped = new_frames.groupby(['Doc Date', 'Branch (Name)', 'PHYID', 'Product (Code)', 'Product (Name)', 'Serial', 'Category Name'], as_index=False, dropna=False).agg({'Number' : 'sum'}).set_index(["PHYID", 'Product (Code)', 'Serial'], drop=False)
grouped_phyidonly = new_frames["PHYID"].unique()

#convert into dictionary (maps) for firestore
nested_dict = lambda: defaultdict(dict)
nested = nested_dict()
nested2 = nested_dict()

total_phyid = grouped["PHYID"].nunique()
total_items = sum(grouped["Number"])

nested2 = {'Total_Bill' : total_phyid, 'Total_Items' : total_items}

for row2 in grouped.itertuples(index=True):
    nested[row2.Index[0]][str(row2.Index[2]).strip(".0")] = {'PHYID' : row2.Index[0], 'Doc Date' : row2._1, 'Branch' : row2._2, 'Product Code' : str(row2.Index[1]), 'Product Name' : row2._5, 'Serial' : str(row2.Index[2]).strip(".0"), 'Number' :row2.Number, 'Category Name' : row2._7}


In [11]:
with pd.ExcelWriter("test.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace",) as excel:
    grouped.to_excel(excel, sheet_name="test")

<H1>SEND DATA

In [ ]:
#send to firestore
print(len(nested))
today = datetime.datetime.today()
batch = firestore_db.batch()
transaction = firestore_db.transaction()

#batch_size = 500
#write data
for number, field in enumerate(grouped_phyidonly):
    if today.day >= 15:
        return49_data = firestore_db.collection("Return49_Data").document("Year").collection(str(today.year)).document("Month").collection(str(today.month)).document(field)
    else:
        return49_data = firestore_db.collection("Return49_Data").document("Year").collection(str(today.year)).document("Month").collection(str(today.month - 1)).document(field)
    if number % 500 == 499:
        batch.commit()
        batch = firestore_db.batch()
    batch.set(return49_data, nested[field],merge=True)
batch.commit()

#write total data
if today.day >= 15:
    return49_data_util = firestore_db.collection("Return49_Data_Util").document("Year").collection(str(today.year)).document("Month").collection(str(today.month)).document("Total_return49_data")
else:
    return49_data_util = firestore_db.collection("Return49_Data_Util").document("Year").collection(str(today.year)).document("Month").collection(str(today.month - 1)).document("Total_return49_data")

@firestore.transactional
def update_in_transaction(transaction, return49_data_util):
    snapshot = return49_data_util.get(transaction=transaction)
    if snapshot.exists:
        transaction.update(return49_data_util, {'Total_Bill' : snapshot.get('Total_Bill') + nested2['Total_Bill'], 'Total_Items' : snapshot.get('Total_Items') + nested2['Total_Items']})
    else: 
        transaction.set(return49_data_util, {'Total_Bill' : nested2['Total_Bill'], 'Total_Items' : nested2['Total_Items']}, merge=True)
update_in_transaction(transaction, return49_data_util)

In [45]:
nested2['Total Bill']

64